In [12]:
import geopandas as gpd
import pandas as pd
import requests
import json

In [13]:
# Load list of current AOI regions
url = 'https://raw.githubusercontent.com/GeoscienceAustralia/dea-ard-scene-select/master/scene_select/data/Australian_AOI.json'
region_dict = json.loads(requests.get(url).text)


In [105]:
# Load Territorial Sea Zone shapefile
tsz_gdf = gpd.read_file("au_tsz_line_april2022.zip").to_crs("EPSG:4326").dissolve()

## Landsat

In [91]:
# Load Landsat WRS2 tiles
landsat_gdf = gpd.read_file('https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/WRS2_descending_0.zip').to_crs("EPSG:4326")

In [109]:
# Find polygons that intersect with TSZ
landsat_tsz_gdf = gpd.sjoin(landsat_gdf, tsz_gdf, predicate='intersects')

In [158]:
# Remove any regions already in the DEA AOI and restrict to non-Antartic
landsat_tsz_nonaoi_gdf = landsat_tsz_gdf.loc[
    (~landsat_tsz_gdf.WRSPR.isin(region_dict["ls"])) & (landsat_tsz_gdf.geometry.centroid.y > -60)
]

/tmp/ipykernel_222/706837659.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  (~landsat_tsz_gdf.WRSPR.isin(region_dict["ls"])) & (landsat_tsz_gdf.geometry.centroid.y > -60)


In [140]:
landsat_tsz_nonaoi_gdf.to_file("landsat_additional_regions.geojson")

## Sentinel-2

In [144]:
!git clone https://github.com/justinelliotmeyers/Sentinel-2-Shapefile-Index.git

Cloning into 'Sentinel-2-Shapefile-Index'...
remote: Enumerating objects: 17, done.
remote: Total 17 (delta 0), reused 0 (delta 0), pack-reused 17
Receiving objects: 100% (17/17), 7.24 MiB | 18.03 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [145]:
# Load Sentinel-2 MRGS tiles
s2_gdf = gpd.read_file('Sentinel-2-Shapefile-Index/sentinel_2_index_shapefile.shp').to_crs("EPSG:4326")

In [146]:
# Find polygons that intersect with TSZ
s2_tsz_gdf = gpd.sjoin(s2_gdf, tsz_gdf, predicate='intersects')

/tmp/ipykernel_222/2960138987.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  s2_tsz_gdf.geometry.centroid.y > -60


35284    False
35293    False
35302    False
35303    False
36230    False
         ...  
53468     True
54429     True
54430     True
55335     True
55336     True
Length: 446, dtype: bool

In [156]:
# Remove any regions already in the DEA AOI and restrict to non-Antartic
s2_tsz_nonaoi_gdf = s2_tsz_gdf.loc[
   (~s2_tsz_gdf.Name.isin(region_dict["s2"])) & (s2_tsz_gdf.geometry.centroid.y > -60)
]

/tmp/ipykernel_222/3912617125.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  (~s2_tsz_gdf.Name.isin(region_dict["s2"])) & (s2_tsz_gdf.geometry.centroid.y > -60)


In [159]:
s2_tsz_nonaoi_gdf.to_file("s2_additional_regions.geojson")

In [161]:
len(region_dict["ls"])

431